# Produce covariances from xs reconstruction rule

In [ ]:
import sandy

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")

Cr-50 from JEFF-3.3 was identified as having covariance matrices (MF33) for the partial components of neutron inelastic scattering - i.e., MT=51-70 - but not for the MT=4.

## Plotting covariance matrices available in ENDF-6 file

In [ ]:
mts = range(51, 56)  # only 5 reactions are selected

In [ ]:
errorr = sandy.get_endf6_file("jeff_33", "xs", 240500).get_errorr(
    err=0.1,
    errorr33_kws=dict(
        mt=mts,
        irespr=0,  # faster
        ek=[1e6, 2e6, 5e6, 1e7, 2e7],  # only above threshold
    ),
    verbose=True,
)
cov = errorr['errorr33'].get_cov()

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6), dpi=100)
ax.set_aspect("equal")
sns.heatmap(data=cov.get_corr().data, ax=ax, cmap="bwr", vmin=-1, vmax=1)
fig.tight_layout()

## Transfering covariance info to random xs samples

In [ ]:
tape = sandy.get_endf6_file("jeff_33", "xs", 240500).get_pendf(err=0.1)
xs = sandy.Xs.from_endf6(tape)

### Sampling cross sections **WITHOUT** reconstructing redundant cross section.

In [ ]:
n = 50
smp = cov.sampling(n)
xs = sandy.Xs.from_endf6(tape)

xs_pert = []
for k, p in smp.iterate_xs_samples():
    x = sandy.xs.xs_perturb_worker(xs, n, p).data.unstack().rename("XS").reset_index().assign(SMP=k)
    xs_pert.append(x)
data = pd.concat(xs_pert, ignore_index=True)

filtered_data = data[data.MT.isin([4, 51, 52, 53])].query("E>7.9E5 & E<2e7")

In [ ]:
g = sns.relplot(
    data=filtered_data,
    x="E",
    y="XS",
    col="MT",
    kind="line",
    height=3, aspect=.7,
    errorbar="sd",
)
g.set(yscale="log", xlim=(7.9e5, 2e7), ylim=(1e-2, 2))
g.figure.set_dpi(100);

In [ ]:
corr = filtered_data.pivot_table(index="SMP", columns=("MT", "E"), values="XS").corr()

fig, ax = plt.subplots(figsize=(6, 6), dpi=100)
sns.heatmap(corr, cmap='bwr', vmin=-1, vmax=1, cbar_kws={'label': 'correlation'})
ax.axhline(98 * 1, color='k')
ax.axvline(98 * 1, color='k')
ax.axhline(98 * 2, color='k')
ax.axvline(98 * 2, color='k')
ax.axhline(98 * 3, color='k')
ax.axvline(98 * 3, color='k')

fig.tight_layout()

### Sampling cross sections **AND** reconstructing redundant cross section.

In [ ]:
n = 50
smp = cov.sampling(n)
xs = sandy.Xs.from_endf6(tape)

xs_pert = []
for k, p in smp.iterate_xs_samples():
    x = sandy.xs.xs_perturb_worker(xs, n, p).reconstruct_sums(drop=True).data.unstack().rename("XS").reset_index().assign(SMP=k)
    xs_pert.append(x)
data = pd.concat(xs_pert, ignore_index=True)

filtered_data = data[data.MT.isin([4, 51, 52, 53])].query("E>7.9E5 & E<2e7")

In [ ]:
g = sns.relplot(
    data=filtered_data,
    x="E",
    y="XS",
    col="MT",
    kind="line",
    height=3, aspect=.7,
    errorbar="sd",
)
g.set(yscale="log", xlim=(7.9e5, 2e7), ylim=(1e-2, 2))
g.figure.set_dpi(100);

In [ ]:
corr = filtered_data.pivot_table(index="SMP", columns=("MT", "E"), values="XS").corr()

fig, ax = plt.subplots(figsize=(6, 6), dpi=100)
sns.heatmap(corr, cmap='bwr', vmin=-1, vmax=1, cbar_kws={'label': 'correlation'})
ax.axhline(98 * 1, color='k')
ax.axvline(98 * 1, color='k')
ax.axhline(98 * 2, color='k')
ax.axvline(98 * 2, color='k')
ax.axhline(98 * 3, color='k')
ax.axvline(98 * 3, color='k')

fig.tight_layout()